# Python: A Further Introduction #

Some further language topics of interest, following on from my basic Python introduction.

This will be using Python 3.8, which is the latest version. You may come across a lot of code requiring Python 2.7, the last of the Python 2 series. Some deliberate backward incompatibilities were introduced in Python 3 to fix problems that could not be handled in a backward-compatible fashion.

Previously covered in part 1:
* Numbers
* Strings
* Lists, mutability
* Dictionaries
* Control constructs: `for`-loop, `if`-statement
* Function definitions
* Sets
* Classes (introductory)

And now, onward ...

## There Are No Declarations In Python ##

Before we go much further, something needs to be made clear about Python syntax: **every statement is an executable (imperative) statement**. This applies to class and function definitions, which you can consider to be special kinds of assignment statement. Classes and functions are first-class objects in Python, and like all objects, they are created at run-time, not compile-time. What is generated at compile-time is only the code, but class and function objects consist not just of code, but also data. This kind of thing is valid in Python:

    if «cond» :

        def my_func(...) :
            ... definition 1 for my_func ...
        #end my_func

    else :

        def my_func(...) :
            ... definition 2 for my_func ...
        #end my_func

    #end if

    ... the definition of my_func in effect here depends on «cond» ...

and also this:

    def my_func(...) :
        ... definition 1 for my_func ...
    #end my_func

    ... definition 1 for my_func in effect here ...

    def my_func(...) :
        ... definition 2 for my_func ...
    #end my_func

    ... definition 2 for my_func in effect here ...

or even this:

    def my_func(...) :
        ... definition 1 for my_func ...
    #end my_func

    ... definition 1 for my_func in effect here ...

    my_func = 2 + 2

    ... my_func no longer references a function here ...

and the same is true of class definitions.

## Reflection/RTTI ##

*Reflection* is a fancy term for being able to examine and manipulate type information at run-time. Or a more limited form of this might be called *Run-Time Type Information* (RTTI). Some languages have a complex system for doing this, usually with only partial functionality. Python, on the other hand, being a fully dynamic language, can offer full access, even being able to do things like create new types at run time.

The `issubclass` built-in function lets you query subclass/superclass relationships. This works among the built-in types as well.

In [ ]:
issubclass(int, float)

In [ ]:
issubclass(bool, int)

To determine the type of an object, you can use the `type` built-in function, as shown in previous examples. Types are objects too, and in particular you can compare them for equality:

In [ ]:
type(3) == int

But if you need to check that a value is of an acceptable type, it is better to use the `isinstance` function, since this will also accept values of subclasses (yes, even the built-in types can be subclassed):

In [ ]:
isinstance(3, int)

In [ ]:
isinstance(False, int)

You can check that a value is of (or a subclass of) any of a list or tuple of types:

In [ ]:
isinstance(3, (int, float)), isinstance(3.0, (int, float))

If you want to check that a value is of a numeric type, the `numbers` module provides *abstract base classes* to make this more convenient:

In [ ]:
import numbers

issubclass(int, numbers.Real), issubclass(float, numbers.Real)

In [ ]:
isinstance(3, float)

In [ ]:
isinstance(3, numbers.Real)

Another set of built-in functions lets you dynamically manipulate attributes: `dir()`, `hasattr()`, `getattr()`, `setattr()`, `delattr()`.

## More Looping Fun ##

Returning to our previous stock-control example, suppose we would like the stock printout to include an item number on each line. One way to do this is as follows:

In [ ]:
stock = {"apples" : 5, "tangeloes" : 3, "pears" : 3}

In [ ]:
def show_itemized_stock() :
    i = 0
    for k in sorted(stock) :
        i += 1
        print("{}. {:.<12}{}".format(i, k, stock[k]))
    #end for
#end show_itemized_stock
show_itemized_stock()

However, Python offers a built-in function called `enumerate`, which makes this a little easier:

In [ ]:
def show_itemized_stock() :
    for i, k in enumerate(sorted(stock)) :
        print("{}. {:.<12}{}".format(i, k, stock[k]))
    #end for
#end show_itemized_stock
show_itemized_stock()

Note that, in common with Python conventions elsewhere, `enumerate()` defaults to numbering things from zero. But it can an optional second arg to indicate the starting point for numbering:

In [ ]:
def show_itemized_stock() :
    for i, k in enumerate(sorted(stock), 1) :
        print("{}. {:.<12}{}".format(i, k, stock[k]))
    #end for
#end show_itemized_stock
show_itemized_stock()

## Iterators & Generators ##

In Python, an _iterable_ is any object type from which you can construct an _iterator_ , using the `iter()` built-in function. An _iterator_ is something that returns successive elements of some sequence, each time you pass it to the `next()` built-in function. For example, lists and tuples are iterables:

In [ ]:
l = [1, 2, 3]
i = iter(l)

In [ ]:
print(next(i))

Note how the end of the sequence is reported by raising the standard `StopIteration` exception. An alternative is to pass some special sentinel value as the second argument of `next()`, which will be returned once the iterator runs out of things to return:

In [ ]:
print(next(i, None))

An iterator is also an iterable; that means that passing it to the `iter()` function returns the same thing:

In [ ]:
i is iter(i)

When you write a `for`-statement, it handles all of this machinery behind the scenes automatically for you: calling `iter()` on the loop expression that you pass, doing `next()` calls each time round the loop and assigning the result to the index variable, and catching `StopIteration` to exit the loop.

You can also pass iterators to the `list()` and `tuple()` constructors; they will collect all the successive values returned from the iterator, and build them into a list or tuple respectively.

One way you can construct your own iterators is by writing a _generator_ function. This is a function that, instead of doing a `return`, contains a `yield` construct instead.

In [ ]:
def my_gen(a) :
    print("started execution")
    yield a + 1
    print("continuing execution")
    yield a + 2
    print("finished execution")
#end my_gen

Calling the function does not actually cause it to execute (yet); it returns a generator object that you can pass to `next()`, as with any iterator.

In [ ]:
f = my_gen(3)
print(f)

The first call to `next()` actually starts the function executing, until it gets to a `yield`, where its execution is suspended and the `yield`ed value is returned.

In [ ]:
print(next(f))

The function execution can be resumed from this point by another call to `next()`:

In [ ]:
print(next(f))

And then, when the function actually returns, a `StopIteration` exception is automatically raised:

In [ ]:
print(next(f))

### Generator Example: Permutations ###

Consider the problem of generating all permutations of a given list, e.g. the list `[1, 2, 3]` has $3! = 6$ permutations:

    [1, 2, 3]
    [1, 3, 2]
    [2, 3, 1]
    [2, 1, 3]
    [3, 1, 2]
    [3, 2, 1]

How do we generate these? The general algorithm can be expressed *recursively* as follows:

* If the list is empty, then there is only one permutation: the empty list.
* Otherwise, pick each element of the list in turn. For each such selection:
  * for each permutation of the remaining items in the list, put the previously-selected element on the front, and return this as a permutation.

This can be expressed in Python as follows:

In [ ]:
def permute(l) :
    if len(l) == 0 :
        yield []
    else :
        for i, elt in enumerate(l) :
            for rest in permute(l[:i] + l[i + 1:]) :
                yield [elt] + rest
            #end for
        #end for
    #end if
#end permute

In [ ]:
p = permute(["a", "b"])

In [ ]:
next(p)

In [ ]:
next(p)

In [ ]:
next(p)

Construct a list of the values, or use our generator in a `for`-loop:

In [ ]:
list(permute([1, 2, 3]))

In [ ]:
for c in permute([1, 2, 3]) :
    print(c)
#end for

What’s the advantage of using a generator? It can be handy to avoid storing the whole of a large list in memory at once, where you only need to process one element at a time. For example, the function might do a database query, and `yield` each matching record, one at a time, and there might be a million matching records.

## Comprehensions ##

Python allows you to write expressions like

    «expression involving «var»» for «var» in «iterable»

Such an expression is called a *comprehension*, and its value is an iterator. Following are some examples of this in action.

In [ ]:
sample_list_1 = [1, 2, 3]
sample_list_2 = [1, "two", 3]

In [ ]:
i1 = (isinstance(i, int) for i in sample_list_2)

In [ ]:
type(i1)

In [ ]:
list(i1)

In [ ]:
i1 = [isinstance(i, int) for i in sample_list_2]
i2 = list(isinstance(i, int) for i in sample_list_2)
i3 = (isinstance(i, int) for i in sample_list_2)
i3 = [i3]
i4 = (isinstance(i, int) for i in sample_list_2)
i4 = list(i4)
print(i1, i2, i3, i4)

## `any` and `all` ##

Supposing you want to check that all elements of a list are of a particular type. Rather than writing a loop statement and collecting the results, you can directly operate on a list comprehension.

In [ ]:
all(isinstance(i, int) for i in sample_list_1), \
all(isinstance(i, int) for i in sample_list_2)

If you wanted the opposite condition, the obvious way would be to put a `not` on the front:

    not all(isinstance(i, int) for i in sample_list_1), \
    not all(isinstance(i, int) for i in sample_list_2)

Another way to express it is using the complementary function to `all`, which is `any`:

In [ ]:
any(not isinstance(i, int) for i in sample_list_1), \
any(not isinstance(i, int) for i in sample_list_2)

In this case, it doesn’t look like one is obviously better than the other. It might be different in other cases.

## Lambdas and `filter()` ##

Supposing you want to extract elements from a list that match some criterion, say you want just the strings from this list:

In [ ]:
items = [2, "green", 3, "two"]

Here’s a long-winded way to do it:

In [ ]:
just_the_strings = []
for item in items :
    if isinstance(item, str) :
        just_the_strings.append(item)
    #end if
#end for
just_the_strings

There is a built-in function called `filter()`, which takes a _predicate_ function and a sequence or iterable as arguments and returns an iterator over the items for which the predicate returns true. We could use it like this:

In [ ]:
def is_string(x) :
    return isinstance(x, str)
#end is_string

just_the_strings = filter(is_string, items)
list(just_the_strings)

Note that coercing the iterator to a `list` forces it to return all its items. But that `is_string` function is probably a “throwaway” function, defined only to be used in this one place. And its body is just `return`ing the value of a single expression. So there’s a way to shorten things even more:

In [ ]:
just_the_strings = filter(lambda x : isinstance(x, str), items)
list(just_the_strings)

In Python, a `lambda` is just a way of defining a function whose body consists of nothing more than the evaluation of a single expression, that can be embedded directly within a containing expression. It’s a convenience, nothing more; it is handy in many places, which is why Python provides it as an additional construct to normal function definitions. You could have written the previous definition of `is_string` like this:

In [ ]:
is_string = lambda x : isinstance(x, str)

but the regular `def` form is usually preferred in this instance, because it gives the function object a name, and also allows the inclusion of a docstring.

## Decorators ##

A *decorator* is a line beginning with “`@`” followed by an expression. These can be used in two places. Prior to a function definition, e.g.

    @«something»
    def func(...) :
        ...
    #end func

which is exactly equivalent to

    def func(...) :
        ...
    #end func
    func = «something»(func)

and prior to a class definition, e.g.

    @«something»
    class MyClass :
        ...
    #end MyClass

which is a shortcut for

    class MyClass :
        ...
    #end MyClass
    MyClass = «something»(MyClass)

What is the point of these? Let us start by considering some built-in functions that are commonly used as decorators for methods inside classes. In other languages (like Java or C++), it is possible to declare methods “static”. That means the method can be invoked via the class name itself, rather than via an instance of the class. Such methods still have access to the innards of the class, even though they have no “`this`” instance that they can reference.

In Python, you can define a static method with the `staticmethod()` built-in function, which can be conveniently used as a decorator:

    class MyClass :

        @staticmethod
        def my_method(arg) :
            ... note there is no arg referencing the current class instance ...
        #end my_method

    #end MyClass

However, because Python has no class visibility controls (e.g. `public`, `private`, `protected`), such a feature is more of a minor convenience for grouping purposes, rather than providing any really important functionality.

What is somewhat more useful is the `classmethod()` function. Whereas a regular method gets the current class instance as its first argument on a call, a classmethod gets the class object itself.

In [ ]:
class ClassMethodExample :

    var = "value for the class"
      # shares a single value across all class instances.

    def __init__(self) :
        self.var = "value for the instance"
          # can have a different value for each instance.
    #end __init__

    def access_from_instance(self) :
        print(type(self))
        return self.var
    #end access_from_instance

    @classmethod
    def access_from_class(cself) :
        print(type(cself))
        return cself.var
    #end access_from_class

#end ClassMethodExample

inst = ClassMethodExample()
print(inst.access_from_instance())
print(inst.access_from_class())
print(ClassMethodExample.access_from_class())
#print(ClassMethodExample.access_from_instance())

Another built-in function called `property()` is useful for defining _properties_. These look like instance variables, but accessing them invokes a method behind the scenes to perform some computation.

There are various ways to use this function. Here it is being used to define a read-only property; note how accessing `sum()` causes a method call, but there are no parentheses following the method name as with a normal call:

In [ ]:
class ROPropertyExample :

    def __init__(self, a, b) :
        self.a = a
        self.b = b
    #end __init__

    @property
    def sum(self) :
        return self.a + self.b
    #end sum

#end ROPropertyExample

inst = ROPropertyExample(2, 3)
print(inst.a, inst.b, inst.sum)
inst.a = 5
print(inst.a, inst.b, inst.sum)

The property as defined is _read-only_ ; attempting to assign to it triggers a run-time error:

In [ ]:
inst.sum = 3

It is possible to define read-write properties as well, by adding a separate setter method that is invoked when the property reference is the LHS of an assignment:

In [ ]:
class RWPropertyExample :

    def __init__(self, a, b) :
        self.a = a
        self.b = b
    #end __init__

    @property
    def sum(self) :
        return self.a + self.b
    #end sum

    @sum.setter
    def sum(self, newsum) :
        self.a = newsum // 2
        self.b = (newsum + 1) // 2
    #end sum

#end RWPropertyExample

inst = RWPropertyExample(2, 3)
print(inst.a, inst.b, inst.sum)
inst.a = 5
print(inst.a, inst.b, inst.sum)
inst.sum = 4
print(inst.a, inst.b, inst.sum)

It is worth noting that while `classmethod()`, `staticmethod()` and `property()` are built-in functions, they are just a convenience; there isn’t actually any “magic” that they do that you cannot do yourself directly in Python. They are examples of the use of a more fundamental underlying Python concept called “descriptors”, that I won’t go into here.

Defining your own decorators: quick look at a more elaborate example of how to define interface classes for D-Bus using my `ravel` module from [DBussy](https://github.com/ldo/dbussy). Look at code for the Big Ben server and clients in the [DBussy Examples](https://github.com/ldo/dbussy_examples) repo.

## Lexical Binding ##

An important aspect of functions and classes being first-class objects is that they can be used in expressions and returned as results from expressions. For example, you can define a function or class inside a function, and return it as a result from that function.

Consider a function that can evaluate any degree-1 polynomial (equation of a straight line):

In [ ]:
def poly1(x, a, b) :
    return a * x + b
#end poly1

Maybe we have specific polynomials that we frequently want to evaluate:

    def poly1_3_5(x) :
        return 3 * x + 5
    #end poly1_3_5

    def poly1_5_8(x) ;
        return 5 * x + 8
    #end poly1_5_8

Rather than having to write out the full formula for the polynomial each time, wouldn’t it save some effort if we could have a function that will create a polynomial-evaluation function for a given polynomial? Going back to the original general `poly1` function, if there were some way we could “freeze” the values of args `a` and `b`, to produce a new function that only needs to be called with a value for `x`.

Like this:

In [ ]:
def genpoly1(a, b) :

    def poly1(x) :
        return a * x + b
    #end poly1

#begin genpoly1
    return poly1
#end genpoly1

poly1_3_5 = genpoly1(3, 5)
poly1_5_8 = genpoly1(5, 8)

poly1_3_5(3), poly1_5_8(4)

Note that `a` and `b` are variables local to the outer `genpoly1` function. What happens to those values after that outer function returns? In fact, they don’t disappear, as long as a reference exists to the inner function that is capable of accessing them.

There is only one copy of the code for each function `genpoly1` and `poly1`. But there is a new function _object_ created for `poly1` each time `genpoly1` is called; this object references the same code, but it has its own instance of the environment that holds the local variables for that call of `genpoly1`. And those variables are _distinct_ from any others with names `a` and `b` that might be present at the point of call, e.g.:

In [ ]:
def not_dynamic_binding() :
    a = 9
    b = 10
    x = 3
    print(a * x + b)
    print(poly1_5_8(x))
#end not_dynamic_binding

not_dynamic_binding()

**Summary:** The core of the Python language can be defined very compactly (I estimate the language reference is about 140 printed pages), certainly compared to other general-purpose languages. Most of the power of the language comes from libraries, both standard ones that come with the language and a whole host of third-party ones. These languages take full advantage of the power of the core, so using them becomes like using a whole lot of additional features built into the language. You can get some flavour of this power from the examples above, but more will become apparent as you delve into the libraries.

Have fun.